In [1]:
import json
import hashlib
from datetime import datetime
import urllib
import pandas as pd
import numpy as np
import math
import datetime as dt
import yfinance as yf

In [2]:
import logging

# Create a logger object
log = logging.getLogger()
log.setLevel(logging.DEBUG)  # Set the root logger level to DEBUG


# Create a console handler for printing logs to console
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)  # Set console handler level to INFO

# Define format for both handlers
formatter = logging.Formatter('[%(asctime)s][%(levelname)s] - %(message)s')
console_handler.setFormatter(formatter)

# Add the handlers to the logger

log.addHandler(console_handler)



In [53]:
import yfinance as yf

# Ticker symbol for ITC
ticker_symbol = "^NSEBANK"  # For the National Stock Exchange of India

# Fetch historical data
data = yf.download(ticker_symbol, period="5d", interval="1m")
data['Change']=data['Close']-data['Open']
data['Pre_Change']= (data['Change']/data['Open'])*100


In [60]:
old_price_long = 0
old_price_short = 0
target_prencentage = 0.01
stop_loss_precentage = 0.3
stop_loss_long = False
stop_loss_short = False
stop_loss_price_long = 0
stop_loss_price_short = 0
long_hold = False
short_hold = False
trade_book = []
qty = 30
brokerage = 20*1.18
other_charges_long = (0.004/100)*1.18
other_charges_short = 0.0149769/100



for index, row in data.iterrows():

    log.debug(f"variables values when running on_ticks getting ticks :- old_price_long : {old_price_long},old_price_short : {old_price_short},target_prencantage : {target_prencentage},stop_loss_precentage : {stop_loss_precentage},long_hold : {long_hold},short_hold : {short_hold},trade_book : {trade_book},qty : {qty}")
    #records_ticks(ticks['datetime'],ticks['open'],ticks['high'],ticks['low'],ticks['close'])

    

    current_price = row['Close']
    date = index
    total_charges_long = current_price*qty*other_charges_long+brokerage
    total_charges_short = current_price*qty*other_charges_short+brokerage

    per_change_long = ((current_price-old_price_long)/old_price_long)*100 if old_price_long != 0 else 0
    per_change_short = ((current_price-old_price_short)/old_price_short)*100 if old_price_short != 0 else 0
    price_change_long = current_price-old_price_long
    price_change_short = current_price-old_price_short
    log.info(f"% Change in Price for Long position = {per_change_long}")
    log.info(f"% Change in Price for Short position = {per_change_short} NOTE :: Negetive means profit in short trade same formula has been used to both long and short") 
    
    if (per_change_long >= target_prencentage) and long_hold :

        log.info(f"Exiting Long Trade since Both the Condition Satisfied per_change_long = {per_change_long} is greater than target_precentage = {target_prencentage} and Holding Position for Long is = {long_hold}")
        long_hold = False
        log.debug(f"setting long_hold to False.value after setting false = {long_hold}")
        profit_long = price_change_long
        log.info(f"Sold at {current_price} * {qty} = {current_price*qty}. Booked Profit of {profit_long} * {qty} = {profit_long*qty}\n")
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'exit long','price':current_price,'order_value':int(order_value),'profit':profit_long*qty,'charges':total_charges_short,'net_profit':profit_long*qty-total_charges_short})
        log.debug(f"Appended to Trade book ('date':{date},'type':'exit long','price':{current_price},'order_value':{int(order_value)},'profit':{profit_long*qty})")

    if per_change_long <= -stop_loss_precentage and long_hold:

        log.warning(f"Stoploss on Long Position Triggered since Both Condition per_change_long = {per_change_long} is greater than Stoploss = {stop_loss_precentage} in absolute values and Long_hold is True = {long_hold}. NOTE per_change_long should be negetive since it is decreasing.")
        long_hold = False
        log.debug(f"Assigned False value to long_hold : {long_hold}")
        stop_loss_long = True
        log.debug(f"Changed stop_loss_long value to True since it is triggered. new value = {stop_loss_long}")
        loss = current_price-old_price_long
        log.warning(f"Exiting Long Position Booked a LOSS of {loss} * {qty} = {loss*qty}\n")
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'exit long STOPLOSS','price':current_price,'order_value':int(order_value),'profit':loss*qty,'charges':total_charges_short,'net_profit':loss*qty-total_charges_short})
        log.debug(f"Appended to Trade book ('date':{date},'type':'exit long STOPLOSS','price':{current_price},'order_value':{int(order_value)},'profit':{loss*qty})")

    if (not long_hold) and (not stop_loss_long):

        log.info(f"Going Long Since not Holding and neither stoploss is triggered values :: long_hold : {long_hold}, stop_loss_long : {stop_loss_long}")
        long_hold = True

        log.info(f'Buying at a price {current_price} * {qty}. total buy price {qty*current_price} \n ')
        old_price_long = current_price
        log.debug(f"Setting global old_price_long to {current_price} and long_hold to {long_hold}")
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'long','price':current_price,'order_value':int(order_value),'profit':0,'charges':total_charges_long,'net_profit':0-total_charges_long})


    
    if (per_change_short <= -target_prencentage) and short_hold:
        log.debug(f"Exiting Short Trade since Both Condition Satisfied. per_change_short : {per_change_short} is greater than target_prencentage : {target_prencentage} and short hold is True Value : {short_hold}")
        short_hold = False
        log.debug(f'Set short_hold to False new Value = {short_hold}')
        profit_short = old_price_short-current_price 
        log.info(f"Exiting Trade at price {current_price} * qty {qty} = {current_price*qty}")
        log.info(f"Booked a profit of {profit_short} * {qty} = {profit_short*qty}")
        order_value = qty*current_price
        
        trade_book.append({'date':date,'type':'exit short','price':current_price,'order_value':int(order_value),'profit':profit_short*qty,'charges':total_charges_long,'net_profit':profit_short*qty-total_charges_long})



    if per_change_short >= stop_loss_precentage and short_hold:
        log.warning(f"Stoploss on Short Position Triggered since per_change_short : {per_change_short} is greater than stop_loss_precentage : {stop_loss_precentage}. in absolute term. and short_hold is True : {short_hold} NOTE :: per_change_should be positive since it is a short trade")
        short_hold = False
        log.debug('Assigned new value to short_hold False : {False}')
        stop_loss_short = True
        log.debug(f'Since Stoploss triggered setting stop_loss_short to Truew new value : {stop_loss_short}')
        loss_on_short = old_price_short-current_price
        log.warning(f'Exiting short Trade at price : {current_price}. Booked a loss of {loss_on_short} *qty = {loss_on_short*qty}')
        
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'exit short Stoploss','price':current_price,'order_value':int(order_value),'profit':loss_on_short*qty,'charges':total_charges_long,'net_profit':loss_on_short*qty-total_charges_long})

    
    if (not short_hold) and (not stop_loss_short):
        
        log.info(f"Going Short since Both short_hold : {short_hold} and stop_loss_short : {stop_loss_short} are False. ")
        short_hold = True
        log.info(f'Shorting at a price {current_price}. total buy price {qty*current_price} \n ')
        old_price_short = current_price
        log.debug(f"Setting old_short_price to {current_price}. value after setting {old_price_short}")
        order_value = qty*current_price
        trade_book.append({'date':date,'type':'short','price':current_price,'order_value':int(order_value),'profit':0,'charges':total_charges_short,'net_profit':0-total_charges_short})

    if index.hour==15 and index.minute==29:
        if long_hold:
            long_hold = False
            profit_long = current_price-old_price_long
            order_value = qty*current_price
            trade_book.append({'date':index,'type':'exit long','price':current_price,'order_value':int(order_value),'profit':profit_long*qty,'charges':total_charges_short,'net_profit':(profit_long*qty)-total_charges_short})


        if short_hold:
            short_hold = False
            profit_short = old_price_short-current_price
            order_value = qty*current_price
            trade_book.append({'date':index,'type':'exit short','price':current_price,'order_value':int(order_value),'profit':profit_short*qty,'charges':total_charges_long,'net_profit':(profit_short*qty)-total_charges_long})
            

        break

[2024-03-23 12:19:43,176][INFO] - % Change in Price for Long position = 0
[2024-03-23 12:19:43,177][INFO] - % Change in Price for Short position = 0 NOTE :: Negetive means profit in short trade same formula has been used to both long and short
[2024-03-23 12:19:43,177][INFO] - Going Long Since not Holding and neither stoploss is triggered values :: long_hold : False, stop_loss_long : False
[2024-03-23 12:19:43,178][INFO] - Buying at a price 46457.75 * 30. total buy price 1393732.5 
 
[2024-03-23 12:19:43,179][INFO] - Going Short since Both short_hold : False and stop_loss_short : False are False. 
[2024-03-23 12:19:43,179][INFO] - Shorting at a price 46457.75. total buy price 1393732.5 
 
[2024-03-23 12:19:43,182][INFO] - % Change in Price for Long position = 0.10730515898854334
[2024-03-23 12:19:43,182][INFO] - % Change in Price for Short position = 0.10730515898854334 NOTE :: Negetive means profit in short trade same formula has been used to both long and short
[2024-03-23 12:19:43,1

In [61]:
trade_book_df = pd.DataFrame(trade_book)
trade_book_df

,date,type,price,order_value,profit,charges,net_profit
0,2024-03-18 09:15:00+05:30,long,46457.750000,1393732,0.000000,89.384174,-89.384174
1,2024-03-18 09:15:00+05:30,short,46457.750000,1393732,0.000000,232.337923,-232.337923
2,2024-03-18 09:16:00+05:30,exit long,46507.601562,1395228,1495.546875,232.561909,1262.984966
3,2024-03-18 09:16:00+05:30,long,46507.601562,1395228,0.000000,89.454764,-89.454764
4,2024-03-18 09:20:00+05:30,exit long,46562.699219,1396880,1652.929688,232.809467,1420.120221
5,2024-03-18 09:20:00+05:30,long,46562.699219,1396880,0.000000,89.532782,-89.532782
6,2024-03-18 09:25:00+05:30,exit long,46611.648438,1398349,1468.476562,233.029399,1235.447163
7,2024-03-18 09:25:00+05:30,long,46611.648438,1398349,0.000000,89.602094,-89.602094
8,2024-03-18 09:25:00+05:30,exit short Stoploss,46611.648438,1398349,-4616.953125,89.602094,-4706.555219
9,2024-03-18 09:36:00+05:30,exit long STOPLOSS,46455.648438,1393669,-4680.000000,232.328480,-4912.328480


In [62]:
print(sum(trade_book_df['profit']))
print(sum(trade_book_df['charges']))
print(sum(trade_book_df['net_profit']))

-4680.0
1610.643086975117
-6290.643086975117


In [63]:
for index,row in data.iterrows():
    print(index,row['Close'])

2024-03-18 09:15:00+05:30 46457.75
2024-03-18 09:16:00+05:30 46507.6015625
2024-03-18 09:17:00+05:30 46509.80078125
2024-03-18 09:18:00+05:30 46525.6015625
2024-03-18 09:19:00+05:30 46537.8984375
2024-03-18 09:20:00+05:30 46562.69921875
2024-03-18 09:21:00+05:30 46554.3515625
2024-03-18 09:22:00+05:30 46547.05078125
2024-03-18 09:23:00+05:30 46530.05078125
2024-03-18 09:24:00+05:30 46551.8984375
2024-03-18 09:25:00+05:30 46611.6484375
2024-03-18 09:26:00+05:30 46641.3984375
2024-03-18 09:27:00+05:30 46628.1484375
2024-03-18 09:28:00+05:30 46620.8984375
2024-03-18 09:29:00+05:30 46635.19921875
2024-03-18 09:30:00+05:30 46596.3515625
2024-03-18 09:31:00+05:30 46637.75
2024-03-18 09:32:00+05:30 46611.05078125
2024-03-18 09:33:00+05:30 46589.1015625
2024-03-18 09:34:00+05:30 46490.6015625
2024-03-18 09:35:00+05:30 46486.3984375
2024-03-18 09:36:00+05:30 46455.6484375
2024-03-18 09:37:00+05:30 46453.94921875
2024-03-18 09:38:00+05:30 46459.1484375
2024-03-18 09:39:00+05:30 46423.80078125
20